In [1]:
import pandas as pd
import cv2
import os

csv_path = r"C:\Users\Administrator\Desktop\tsi-zenML\zenML---aws-objdet\may15annos.csv"
df = pd.read_csv(csv_path)
df

,Unnamed: 0,local_filepath,filename,label,xmin,xmax,ymin,ymax,pipe_id,dt,side,passnum,cam,video_name,frame
0,0,c:\Users\Administrator\Desktop\notebooks-for-m...,1000_20240515152544_BOX_1_0_11_blue_tape.png,blue_tape,660,995,1893,2144,1000,20240515152544,BOX,1,0,1000_20240515152544_BOX_1_0_11_blue_tape.png,11
1,1,c:\Users\Administrator\Desktop\notebooks-for-m...,1000_20240515152552_BOX_1_0_20_blue_tape.png,blue_tape,1079,1415,1862,2113,1000,20240515152552,BOX,1,0,1000_20240515152552_BOX_1_0_20_blue_tape.png,20
2,2,c:\Users\Administrator\Desktop\notebooks-for-m...,1000_20240515152600_BOX_1_0_30_blue_tape.png,blue_tape,1534,1870,1888,2138,1000,20240515152600,BOX,1,0,1000_20240515152600_BOX_1_0_30_blue_tape.png,30
3,3,c:\Users\Administrator\Desktop\notebooks-for-m...,1000_20240515152608_BOX_1_0_40_blue_tape.png,blue_tape,2005,2341,2015,2266,1000,20240515152608,BOX,1,0,1000_20240515152608_BOX_1_0_40_blue_tape.png,40
4,4,c:\Users\Administrator\Desktop\notebooks-for-m...,1000_20240515152617_BOX_1_0_44_blue_tape.png,blue_tape,2159,2494,2067,2317,1000,20240515152617,BOX,1,0,1000_20240515152617_BOX_1_0_44_blue_tape.png,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518,518,c:\Users\Administrator\Desktop\notebooks-for-m...,1000_20240515164026_BOX_1_0_222_blue_tape.png,blue_tape,209,658,1105,1356,1000,20240515164026,BOX,1,0,1000_20240515164026_BOX_1_0_222_blue_tape.png,222
519,519,c:\Users\Administrator\Desktop\notebooks-for-m...,1000_20240515164034_BOX_1_0_232_blue_tape.png,blue_tape,803,1251,1100,1350,1000,20240515164034,BOX,1,0,1000_20240515164034_BOX_1_0_232_blue_tape.png,232
520,520,c:\Users\Administrator\Desktop\notebooks-for-m...,1000_20240515164042_BOX_1_0_242_blue_tape.png,blue_tape,1391,1840,1100,1350,1000,20240515164042,BOX,1,0,1000_20240515164042_BOX_1_0_242_blue_tape.png,242
521,521,c:\Users\Administrator\Desktop\notebooks-for-m...,1000_20240515164051_BOX_1_0_245_blue_tape.png,blue_tape,1453,1901,1110,1361,1000,20240515164051,BOX,1,0,1000_20240515164051_BOX_1_0_245_blue_tape.png,245


In [2]:
### use this to store current labels
import numpy as np

def one_hot_json():
    """make list of items of interest"""

    categories = [
        {'class_id' : 0, 'name' : 'blue_tape'},
        {'class_id' : 1, 'name' : 'black_tape'},
        {'class_id' : 2, 'name' : 'gum'},
        {'class_id' : 3, 'name' : 'leaf'}
        ]
    
    return categories

def search(name, x):
    
    if len([e for e in x if e['name'] == name]) > 0:
        return [e for e in x if e['name'] == name][0]['class_id']  
    else:
        return np.NaN 

def one_hot_label(label):
    """return class_id from label name"""
    cats = one_hot_json()
    return search(label,cats)

print('type of one hot json: ', type(one_hot_json()))

x = one_hot_label('black_tape')
print('x ', x)

x = one_hot_label('gum')
print('x ', x)

x = one_hot_label('leaf')
print('x ', x)

x = one_hot_label('yarg')
print('x ', x)


type of one hot json:  <class 'list'>
x  1
x  2
x  3
x  nan


In [3]:
## images and jsons are in S3 buckets, 
# test channels to see if training spools up
images_bucket = 'tape-exp-images-may30'
annos_bucket = 'tape-exp-annos-may30'

# same buckets for now, change later!
s3_train_images = 's3://{}/'.format(images_bucket)
s3_val_images = 's3://{}/'.format(images_bucket)
s3_train_anno = 's3://{}/'.format(annos_bucket)
s3_val_anno = 's3://{}/'.format(annos_bucket)

In [6]:
import sagemaker
from sagemaker import image_uris
from sagemaker.workflow.pipeline_context import PipelineSession
import boto3

# 4. invoke sagemaker, upload rec files to aws

sagemakerRoleName = 'SkylerSageMakerRole'
bucket = 'tubes-tape-exp-models'
model_framework = "object-detection"
prefix = 'retinanet'
model_bucket_path = "s3://tubes-tape-exp-models/"
s3_output_location = "s3://{}/{}/output".format(bucket,prefix)
inst_type = "ml.p3.8xlarge" #<---costly, but fast. 21 mins per 100 epochs for 500img dataset
region = 'us-east-1'

sess = sagemaker.Session()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName=sagemakerRoleName)['Role']['Arn']

training_image = image_uris.retrieve(
    region = sess.boto_region_name,
    framework = model_framework,
    version="1"
)

#print('Sagemaker session :', sess)
print('default bucket :', bucket)
print('Prefix :', prefix)
print('Region selected :', region)
print('IAM role :', role)
print(training_image)

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\Administrator\AppData\Local\sagemaker\sagemaker\config.yaml


Couldn't call 'get_role' to get Role ARN from role name anothersky to get Role path.


default bucket : tubes-tape-exp-models
Prefix : retinanet
Region selected : us-east-1
IAM role : arn:aws:iam::533266972570:role/SkylerSageMakerRole
825641698319.dkr.ecr.us-east-2.amazonaws.com/object-detection:1


In [7]:
model_bucket_path = "s3://tubes-tape-exp-models/"
training_image = image_uris.retrieve(

    region = sess.boto_region_name, framework = "object-detection", version="1"
)

print(training_image)

"""
define pipeline bucket 
"""
pipeline_session = PipelineSession(default_bucket = model_bucket_path)
print(pipeline_session)

825641698319.dkr.ecr.us-east-2.amazonaws.com/object-detection:1


In [8]:
    
train_data_images = sagemaker.inputs.TrainingInput(
    s3_train_images,
    distribution="FullyReplicated",
    content_type="application/x-image",
    s3_data_type="S3Prefix",
)
validation_data_images = sagemaker.inputs.TrainingInput(
    s3_val_images,
    distribution="FullyReplicated",
    content_type="application/x-image",
    s3_data_type="S3Prefix",
)

train_data_annotations = sagemaker.inputs.TrainingInput(
    s3_train_anno,
    distribution="FullyReplicated",
    content_type="application/x-image",
    s3_data_type="S3Prefix",
)
validation_data_annotations = sagemaker.inputs.TrainingInput(
    s3_val_anno,
    distribution="FullyReplicated",
    content_type="application/x-image",
    s3_data_type="S3Prefix",
)

print('data channels here!!')
data_channels = {
    "train": train_data_images, 
    "validation": validation_data_images,
    "train_annotation" : train_data_annotations,
    "validation_annotation" : validation_data_annotations
    }


data channels here!!


In [9]:
num_classes = 4
num_training_samples = 550

num_epochs = 50
lr_steps = '33,67'
base_network ='resnet-50'   
mini_batch_size = 64
lr = 0.0002
lrsf = 0.1
opt = 'adam'
momentum = 0.9
weight_decay = 0.0005
overlap_threshold=0.5
nms_threshold=0.45
image_shape=512
label_width=350

print('num classes: {}, num training images: {}'.format(num_classes, num_training_samples))

#od_mdl = sagemaker_define_model(role, inst_type, training_image, s3_output_location)
od_mdl = sagemaker.estimator.Estimator(
    
    training_image,
    role,
    instance_count = 1,
    instance_type = inst_type,
    volume_size = 50,
    max_run = 360000,
    input_mode = "File",
    output_path = s3_output_location,
    sagemaker_session = sess
)

od_mdl.set_hyperparameters(base_network=base_network,
                                use_pretrained_model=1,
                                num_classes=num_classes,
                                mini_batch_size=mini_batch_size,
                                epochs=num_epochs,               
                                learning_rate=lr, 
                                lr_scheduler_step=lr_steps,      
                                lr_scheduler_factor=lrsf,
                                optimizer=opt,
                                momentum=momentum,
                                weight_decay=weight_decay,
                                overlap_threshold=overlap_threshold,
                                nms_threshold=nms_threshold,
                                image_shape=image_shape,
                                label_width=label_width,
                                num_training_samples=num_training_samples)

# 6. kickoff training

od_mdl.fit(inputs=data_channels, logs=True)

#sagemaker_run_training(od_mdl,data_channels)
print('model is kicked off in aws!')

INFO:sagemaker:Creating training-job with name: object-detection-2024-05-31-20-05-06-735


num classes: 4, num training images: 550
2024-05-31 20:05:07 Starting - Starting the training job......
2024-05-31 20:05:45 Starting - Preparing the instances for training...
2024-05-31 20:06:20 Downloading - Downloading input data...
2024-05-31 20:07:05 Downloading - Downloading the training image.........
2024-05-31 20:08:46 Training - Training image download completed. Training in progress.....Docker entrypoint called with argument(s): train
Running default environment configuration script
Nvidia gpu devices, drivers and cuda toolkit versions (only available on hosts with GPU):
Fri May 31 20:09:16 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.08             Driver Version: 535.161.08   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf  

In [10]:


"""("left", "top", "width", "height"). The "left" (x-coordinate) and "top" (y-coordinate) 
    values represent the upper-left corner of the bounding box. 
    The "width" (x-coordinate) and "height" (y-coordinate) values represent the dimensions of the bounding box. 
"""

import json
import boto3
import matplotlib.pyplot as plt

s3 = boto3.resource('s3')

images_bucket = 'tape-exp-images-may30'
annos_bucket = 'tape-exp-annos-may30'

for i, row in df.iterrows():
        label = row['label']
        
        clss_lbl = one_hot_label(label)
        img_pth = row['local_filepath']
        s3_name = row['filename']
        
        img_r = cv2.imread(img_pth)
        img = cv2.cvtColor(img_r, cv2.COLOR_BGR2RGB) # convert to correct color sequence    

        h, w, c, = img.shape
        
        x_min = float(row['xmin'])
        x_max = float(row['xmax'])
        y_min = float(row['ymin'])
        y_max = float(row['ymax'])
        
        left = x_min
        top = y_max
        width = x_max - x_min
        height = y_max - y_min
        
        #s3_img_path = 'mybucket' + img_pth
        
        filename = os.path.basename(img_pth)
        
        print('filename: ', filename)
        
        s = filename.split('.')
        
        anno = {
            'file' : filename,
            'image_size': [
                {'width' : w, 'height' : h, 'depth': c }
            ],
            'categories' : one_hot_json(),
            
            'annotations' : [
                {
                    "class_id": clss_lbl,
                    "left": left,
                    "top": top,
                    "width": width,
                    "height": height
                    }
                ]
        }
        
        print(anno)
                
        #### ----- PLOT HERE TO VERIFY ---- ######
        # start_point = (int(left),int(top)) # top left
        # end_point = (int(left+width),int(y_min)) # bottom right
        #         # add rect
        # color = (0, 0, 255) # Blue color in RGB
        # thickness = 10 # Line thickness
        # image = cv2.rectangle(img, start_point, end_point, color, thickness)
        # plt.figure(figsize=(3,3))
        # plt.imshow(image)
        # plt.title(label)
        # plt.show()

        
        # save anno to s3
        
        anno_filename = s[0] +'.json'
        s3JSONobject = s3.Object(annos_bucket, anno_filename)
        s3JSONobject.put(
            Body=(bytes(json.dumps(anno).encode('UTF-8')))
            )
                
        #send image to s3
        s3.Bucket(images_bucket).upload_file(img_pth, filename)
        print('file saved to s3!', filename)
                
                
        
        
        

filename:  1000_20240515152544_BOX_1_0_11_blue_tape.png
{'file': '1000_20240515152544_BOX_1_0_11_blue_tape.png', 'image_size': [{'width': 2456, 'height': 3070, 'depth': 3}], 'categories': [{'class_id': 0, 'name': 'blue_tape'}, {'class_id': 1, 'name': 'black_tape'}, {'class_id': 2, 'name': 'gum'}, {'class_id': 3, 'name': 'leaf'}], 'annotations': [{'class_id': 0, 'left': 660.0, 'top': 2144.0, 'width': 335.0, 'height': 251.0}]}
file saved to s3! 1000_20240515152544_BOX_1_0_11_blue_tape.png
filename:  1000_20240515152552_BOX_1_0_20_blue_tape.png
{'file': '1000_20240515152552_BOX_1_0_20_blue_tape.png', 'image_size': [{'width': 2456, 'height': 3070, 'depth': 3}], 'categories': [{'class_id': 0, 'name': 'blue_tape'}, {'class_id': 1, 'name': 'black_tape'}, {'class_id': 2, 'name': 'gum'}, {'class_id': 3, 'name': 'leaf'}], 'annotations': [{'class_id': 0, 'left': 1079.0, 'top': 2113.0, 'width': 336.0, 'height': 251.0}]}
file saved to s3! 1000_20240515152552_BOX_1_0_20_blue_tape.png
filename:  1000

In [ ]:
!pip install label_studio_sdk
!pip install python-dotenv
!pip install opencv-python

In [ ]:

from label_studio_sdk import Client
import dotenv
import os
import json
import pandas as pd
import numpy as np
import boto3
#import matplotlib.pyplot as plt
from datetime import datetime
import cv2
from typing_extensions import Annotated
from typing import Tuple

# from zenml import step
# from zenml.logger import get_logger

dotenv.load_dotenv()

#logger = get_logger(__name__)

API_ROOT = os.getenv("LS_API_ROOT")
LS_API_TOKEN = os.getenv("LS_API_TOKEN")

id = "1"
exportType = "JSON"
output_filename = 'may14annotations.json'
csv_path = os.getcwd() + "\\"+"may15annos.csv"

# image must be saved locally to geenrate binary from img2rec.py

def export_labelstudio_video_annos_csv(ls,id,csv_path):
    
    # csv
    project = ls.get_project(id)
    project.export_tasks(export_type='CSV', export_location=csv_path)

def export_labelstudio_video_annotations(ls,id):
    """generate tasks JSON to create dataset"""
    # Get the project
    project = ls.get_project(id)

    # Export tasks
    tasks = project.export_tasks(export_type='JSON', download_all_tasks=True)
    return tasks

#@step
def dataloader_labelstudio():
    """
    pass label studio tasks JSON object, return dataframe 
    task is JSON video label
    dataframe will have: 
    'local_filepath','filename','label','xmin','xmax','ymin','ymax','pipe_id','dt','side','passnum','cam','video_name','frame'
    ### images saved locally and to AWS s3 bucket
    """
    # Initialize the Label Studio SDK client with the additional headers
    ls = Client(url=API_ROOT, api_key=LS_API_TOKEN)
    tasks = export_labelstudio_video_annotations(ls,id)
    
    target = "fixtargetlater"
    random_state = "999"
    img_data_savepath =  os.getcwd() + "\\"+ "May15-tape-exp-data"
    s3_bucket_savedimages = "tape-experiment-april6"
    
    annotations = []
    cols = ['local_filepath','filename','label','xmin','xmax','ymin','ymax','pipe_id','dt','side','passnum','cam','video_name','frame']

    for i, job in enumerate(tasks): # each task is a labeled video
        
        annos = job['annotations']
        data = job['data']
            
        if len(annos) == 1: # video is annotated
                    
            vid_s3_path = data['video']
            print(f"viewing annos for: {vid_s3_path} ")
            
            # @TODO: flesh out info from vid name...
            pipe_id = '1000'
            side = 'BOX'
            passnum = '1'
            cam = '0'
            
            r = annos[0] # annos stored in list, so remove list              
            n = r['id'] # number of defects persistent during video
                    
            res = r['result']
            
            for j, defect in enumerate(res): # each defect persists by frame
                
                v = defect['value']
                label = v['labels'][0].replace(' ','_')            
                fc = v['framesCount']
                duration = v['duration']
                sequence = v['sequence']
    
                for k, obj in enumerate(sequence):
                                    
                    frame = obj["frame"]
                    x = obj["x"]
                    y = obj["y"]
                    width = obj["width"]
                    height = obj["height"]
                    
                    #load video from S3
                    s3 = boto3.client("s3")
                    bucket_name = "tsi-inc"
                    video_key = vid_s3_path.split('/')[-1]
                    
                    url = s3.generate_presigned_url( ClientMethod='get_object', Params={ 'Bucket': bucket_name, 'Key': video_key } )
                    
                    video = cv2.VideoCapture(url)
                    video.set(cv2.CAP_PROP_POS_FRAMES, frame)
                    ret, f_p = video.read()
                                    
                    if f_p is not None: # need to make sure image is made from video
                        
                        f = cv2.cvtColor(f_p, cv2.COLOR_BGR2RGB) # convert to correct color sequence    
                        img = np.asarray(f)
                        
                        h, w, c = img.shape
                    
                        #resize bounding box locations.. they are in percentage
                        x = (x/100)*w
                        y = (y/100)*h
                                            
                        fudge = 35 # need to confirm this                    
                        height = fudge*height
                        width = fudge*width                
                        
                        # see image with bounding box
                        
                        # add rect
                        color = (0, 0, 255) # Blue color in RGB
                        thickness = 10 # Line thickness 
                        
                        start_point = (int(x),int(y+height)) # top left
                        end_point = (int(x+width),int(y)) # bottom right
                        
                        image = cv2.rectangle(img, start_point, end_point, color, thickness)
                        
                        xmin = int(x)
                        xmax = int(x + width)
                        ymin =int(y)
                        ymax = int(y + height)
                        
                        # use to plot for verificiation, its good...
                        
                        # plt.figure(figsize=(3,3))
                        # plt.imshow(image)
                        # plt.title(label)
                        # plt.show()
                        
                        # save locally, send to S3
                        
                        dt = datetime.today().strftime('%Y%m%d%H%M%S')
                        
                        #filename = f"{pipe_id}_{dt}_{side}_{passnum}_{cam}_{frame}_{label}.png"
                        filename = f"{i}_{j}_{side}_{passnum}_{cam}_{frame}_{label}.png"

                        print('filename: ', filename)
                        local_filepath = img_data_savepath + '\\' + filename
                        #cv2.imwrite(local_filepath, f_p) # save image locally

                        # send image to s3
                        s3b = boto3.resource('s3')
                        #s3b.Bucket(s3_bucket_savedimages).upload_file(local_filepath, filename)
                        #print(f"file saved to s3! {local_filepath} ")
                        
                        # add image, annotation, label, to dataframe 
                        annotations.append([local_filepath,filename,label,xmin,xmax,ymin,ymax,pipe_id,dt,side,passnum,cam,filename,frame])
                        # make df
                        df = pd.DataFrame(data=annotations, columns=cols)
                        
                    else:
                        #print('ERROR  with frame for: ', vid_s3_path)
                        print(f"ERROR with frame {frame} for {vid_s3_path}")
                    
    # make df
    print("dataframe generated!")

# run it

dataloader_labelstudio()

In [ ]:
n_unique = df['label'].nunique()
print(f'number of unique images: {n_unique}')


In [ ]:
df_g = df.groupby(['label'])


In [ ]:
print(df_g)